In [0]:
# First Download the dataset from here https://www.kaggle.com/kumargh/pimaindiansdiabetescsv#pima-indians-diabetes.csv
import pandas as pd
import numpy as np

In [0]:
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
data = pd.read_csv('pima-indians-diabetes.csv', names = names)

In [8]:
data.head()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
# descrribe the dataset
data.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [12]:
# no missing data
data.isnull().sum()

n_pregnant                 0
glucose_concentration      0
blood_pressuer (mm Hg)     0
skin_thickness (mm)        0
serum_insulin (mu U/ml)    0
BMI                        0
pedigree_function          0
age                        0
class                      0
dtype: int64

In [13]:
# Drop rows with missing values if in case any
data.dropna(inplace=True)

# summarize the number of rows and columns in df
data.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [15]:
# Convert dataframe to numpy array
dataset = data.values
print(dataset.shape)

(768, 9)


In [0]:
# split into input (X) and an output (Y)
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [17]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(768, 8)
(768,)
[1 0 1 0 1]


In [0]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [19]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,2.544261e-17,3.614007e-18,-1.327244e-17,7.994184e-17,-3.556183e-17,2.295979e-16,2.398978e-16,1.857600e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00


In [20]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

Using TensorFlow backend.


In [21]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# define a random seed
seed = 6
np.random.seed(6)

In [0]:
# initializing the model with Keras Classifier
model = KerasClassifier(build_fn=create_model,verbose=0)

In [0]:
# define the grid search parameters
batch_size = [10, 20, 30] # number of steps to take while updating the gradient parameters
epochs = [10, 20, 30]

In [0]:
# make a dictionary of the grid search parameters
grid_params = {'batch_size':batch_size,'epochs':epochs}

In [29]:
# build and fit the GridSearchCV
grid = GridSearchCV(estimator=model,param_grid = grid_params,cv=KFold(random_state=seed),verbose=10)
grid_search = grid.fit(X_standardized,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................
[CV] ............ batch_size=10, epochs=10, score=0.740, total=   4.5s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.727, total=   2.8s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.2s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   31.0s remaining:    0.0s


[CV] ............ batch_size=10, epochs=20, score=0.824, total=   5.0s
[CV] batch_size=10, epochs=20 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   36.0s remaining:    0.0s


[CV] ............ batch_size=10, epochs=20, score=0.771, total=   4.9s
[CV] batch_size=10, epochs=30 ........................................
[CV] ............ batch_size=10, epochs=30, score=0.740, total=   7.1s
[CV] batch_size=10, epochs=30 ........................................
[CV] ............ batch_size=10, epochs=30, score=0.721, total=   7.0s
[CV] batch_size=10, epochs=30 ........................................
[CV] ............ batch_size=10, epochs=30, score=0.766, total=   7.1s
[CV] batch_size=10, epochs=30 ........................................
[CV] ............ batch_size=10, epochs=30, score=0.824, total=   7.1s
[CV] batch_size=10, epochs=30 ........................................
[CV] ............ batch_size=10, epochs=30, score=0.752, total=   7.3s
[CV] batch_size=20, epochs=10 ........................................
[CV] ............ batch_size=20, epochs=10, score=0.727, total=   1.9s
[CV] batch_size=20, epochs=10 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.6min finished


In [30]:
# summarize the results
print("Best: {0}, using {1}".format(grid_search.best_score_, grid_search.best_params_))

Best: 0.7735251665115357, using {'batch_size': 30, 'epochs': 10}


In [32]:
# finding mean and standard deviation
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

0.7617774486541748 (0.026676531842905738) with: {'batch_size': 10, 'epochs': 10}
0.765707504749298 (0.03585175445243406) with: {'batch_size': 10, 'epochs': 20}
0.7604872345924377 (0.03484707964060877) with: {'batch_size': 10, 'epochs': 30}
0.7579068064689636 (0.036093386495441054) with: {'batch_size': 20, 'epochs': 10}
0.7670401573181153 (0.05641255360529237) with: {'batch_size': 20, 'epochs': 20}
0.7604787349700928 (0.04684632817461764) with: {'batch_size': 20, 'epochs': 30}
0.7735251665115357 (0.04266636153256992) with: {'batch_size': 30, 'epochs': 10}
0.7423648238182068 (0.07218386576628581) with: {'batch_size': 30, 'epochs': 20}
0.7683303713798523 (0.037286673730105066) with: {'batch_size': 30, 'epochs': 30}


 <h1>Grid Search for DropOut and Learning Rate</h1>

In [0]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

In [0]:
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

In [36]:
# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param)) 

Best: 0.764417290687561, using {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.763118588924408 (0.03971927065385651) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7396146416664123 (0.02014312516041011) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7200916886329651 (0.025276069422086586) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.764417290687561 (0.04262142433431348) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7319412708282471 (0.06718432066749278) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7266021728515625 (0.026242483865176202) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7578983187675477 (0.041658499402744895) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7540022134780884 (0.03678098293284524) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7292420148849488 (0.0355042525386449) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


<h1>Grid search to optimize kernel initialization and activation functions</h1>

In [38]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear'] # most common activations
init = ['uniform', 'normal', 'zero'] #kernel initializer

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] activation=softmax, init=uniform ................................
[CV] .... activation=softmax, init=uniform, score=0.747, total=  12.5s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.5s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.708, total=  11.7s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.1s remaining:    0.0s


...

[CV] ..... activation=softmax, init=normal, score=0.837, total=  11.9s
[CV] activation=softmax, init=normal .................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.8min remaining:    0.0s


[CV] ..... activation=softmax, init=normal, score=0.758, total=  11.9s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.649, total=  12.0s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.584, total=  11.9s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.630, total=  11.5s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.745, total=  12.2s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.647, total=  12.3s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.747, total=  11.5s
[CV] activation=relu, init=uniform ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 11.7min finished


Best: 0.7735081911087036, using {'activation': 'tanh', 'init': 'normal'}
0.7631016135215759 (0.03981837700345801) with: {'activation': 'softmax', 'init': 'uniform'}
0.7592139959335327 (0.04308584587002784) with: {'activation': 'softmax', 'init': 'normal'}
0.6511586427688598 (0.05244526932680711) with: {'activation': 'softmax', 'init': 'zero'}
0.7683388590812683 (0.046181102962835584) with: {'activation': 'relu', 'init': 'uniform'}
0.7670146942138671 (0.042224265288079245) with: {'activation': 'relu', 'init': 'normal'}
0.6511586427688598 (0.05244526932680711) with: {'activation': 'relu', 'init': 'zero'}
0.7696035981178284 (0.03797102405611401) with: {'activation': 'tanh', 'init': 'uniform'}
0.7735081911087036 (0.04012275737227138) with: {'activation': 'tanh', 'init': 'normal'}
0.6511586427688598 (0.05244526932680711) with: {'activation': 'tanh', 'init': 'zero'}
0.7722094893455506 (0.03323642663383448) with: {'activation': 'linear', 'init': 'uniform'}
0.7722179770469666 (0.03882881098465

<h4>So After Tuning the Hyperparameters with Grid Search we see that 'batch_size': 30, 'epochs': 10 'dropout_rate': 0.1, 'learn_rate': 0.001 'activation': 'tanh', 'init': 'normal' are the best parameters to train this model on diabetes Dataset</h4>

In [0]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [0]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))